# Azure AI Agents with Semantic Kernel - Complete Beginner's Tutorial

🎯 **Welcome to your first Azure AI Agents experience using Semantic Kernel!**

This tutorial will guide you through everything you need to know to get started with Azure AI Agents using the Semantic Kernel SDK. We'll cover:

1. **What are Azure AI Agents?** - Understanding the basics
2. **Setup and Prerequisites** - Getting your environment ready
3. **Creating Your First Agent** - Step-by-step agent creation
4. **Simple Conversations** - Basic agent interactions
5. **Adding Plugins** - Extending agent capabilities with custom functions
6. **Advanced Features** - Streaming and complex interactions
7. **Best Practices** - Cleanup and production patterns

**No prior experience required!** We'll start from the very beginning and build up your knowledge step by step.

---

## 🤖 What are Azure AI Agents with Semantic Kernel?

Think of Azure AI Agents with Semantic Kernel as your **AI-powered assistants** that can:
- Have conversations with users
- Remember what was discussed (through "threads")
- Use plugins and tools to extend their capabilities
- Maintain context across multiple interactions
- Leverage the power of Semantic Kernel's orchestration

### Key Concepts (Don't worry, we'll see these in action!):

- **Agent**: Your AI assistant powered by Azure AI and orchestrated by Semantic Kernel
- **Thread**: A conversation session (like a chat conversation)
- **Message**: Individual messages within a conversation
- **Plugin**: Additional capabilities your agent can use (like calling APIs or processing data)
- **Client**: The connection to Azure AI services

### Why Semantic Kernel?
- **Rich Plugin System**: Easily extend agent capabilities
- **Async Support**: Better performance for real-world applications
- **Enterprise Ready**: Built for production scenarios
- **Cross-Platform**: Works across different platforms and languages

Let's dive in! 🚀

## 📋 Prerequisites and Setup

Before we start coding, you'll need:

### 1. Azure AI Services
- An Azure subscription
- An Azure OpenAI resource or Azure AI Services resource
- A deployed model (like GPT-4, GPT-3.5-turbo, etc.)

### 2. Environment Variables or Configuration
You can configure authentication in several ways:
- **Azure Default Credentials** (recommended for Azure-hosted apps)
- **Environment variables** for local development
- **Direct configuration** in code (for learning purposes)

### 3. Python Packages
We'll install the required Semantic Kernel packages in the next cell.

### 4. Azure CLI installed and logged in

**Don't worry if this seems overwhelming - we'll guide you through each step!**

![Single Agent with SK Wrapper](images/single_agent_with_sk.gif)

In [ ]:
# If you haven't created the environment and installed the requirements, install the required packages
# This might take a minute or two

# !pip install azure-ai-agents azure-identity

# Check if the required packages are installed
import importlib.metadata
for package in ["semantic-kernel", "azure-identity"]:
    try:
        version = importlib.metadata.version(package)
        print(f"✅ {package} is installed (version {version}).")
    except importlib.metadata.PackageNotFoundError:
        print(f"❌ {package} is NOT installed.")

In [ ]:
# Let's import everything we need and set up our environment
import asyncio
import os
from typing import Annotated

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.functions import kernel_function

print("📦 All packages imported successfully!")
print("\n🔧 Environment check:")

# Check if we're in a Jupyter environment for async handling
try:
    # Check if we're in Jupyter
    get_ipython()
    print("✅ Jupyter environment detected - async code will work properly")
except NameError:
    print("ℹ️  Running outside Jupyter - using asyncio.run() for async functions")

print("\n🎉 Setup complete! Ready to create your first agent.")

In [ ]:
# Configuration for Azure AI Services
# You can set these environment variables or modify them directly here for learning

# Option 1: Use environment variables (recommended for production)
# os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://your-resource.openai.azure.com/'
# os.environ['AZURE_OPENAI_API_KEY'] = 'your-api-key-here'  # Only if not using managed identity
# os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = 'your-deployment-name'

# Option 2: Direct configuration (for learning and testing)
# We'll use Azure's default settings which work with properly configured Azure resources

print("🔧 Configuration Options:")
print("1. Azure Default Credentials (recommended) - automatically detects Azure authentication")
print("2. Environment variables - set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT_NAME, AZURE_OPENAI_API_KEY for semantic kernel")
print("3. Environment variables - set PROJECT_ENDPOINT, MODEL_DEPLOYMENT_NAME for Foundry SDK")
print("4. Direct configuration - modify the code above")
print("\n✅ Using Azure Default Credentials for this tutorial")
print("ℹ️  This works automatically when running in Azure or when logged in via Azure CLI")

In [ ]:
required_vars = ['AZURE_OPENAI_ENDPOINT', 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_DEPLOYMENT_NAME', 'PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME']
missing_vars = []

for var in required_vars:
    if var not in os.environ:
        missing_vars.append(var)
    else:
        print(f"✅ {var} is set")

if missing_vars:
    print(f"\n❌ Missing environment variables: {missing_vars}")
    print("\n🔧 Please set them using:")
    for var in missing_vars:
        print(f"   os.environ['{var}'] = 'your_value_here'")
else:
    print("\n🎉 All environment variables are properly configured!")

In [ ]:
# Check if Azure CLI is logged in
import subprocess
import shutil
import json

def is_az_logged_in():
    az_path = shutil.which("az")
    if not az_path:
        print("❌ Azure CLI (az) not found in PATH.")
        return False

    try:
        result = subprocess.run(
            [az_path, "account", "show", "--output", "json"],
            capture_output=True,
            text=True,
            check=True
        )
        account_info = json.loads(result.stdout)
        print(f"✅ Logged in as: {account_info['user']['name']}")
        return True
    except subprocess.CalledProcessError:
        print("❌ Not logged in or error during az call. Please log in using 'az login' from the terminal.")
        return False

_ = is_az_logged_in()


## 🔐 Step 1: Create Your Azure AI Agent Client

With Semantic Kernel, we create an **AzureAIAgent** that handles both the connection to Azure AI services and the agent logic. This is different from the Foundry SDK approach - here we create the agent directly!

**What's happening here:**
- We connect to your Azure AI services using Azure's default authentication
- We create an agent definition on the Azure AI service
- We wrap it in a Semantic Kernel AzureAIAgent for enhanced capabilities
- This agent will be our main interface for conversations

In [ ]:
# Create our Azure AI Agent using Semantic Kernel
# This approach creates the agent directly with all capabilities included

async def create_azure_ai_agent():
    """
    Creates an Azure AI Agent using Semantic Kernel.
    This function demonstrates the modern approach to agent creation.
    """
    try:
        model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
        endpoint = os.environ.get("PROJECT_ENDPOINT")

        # Create credentials and client
        client = AzureAIAgent.create_client(credential=DefaultAzureCredential(), endpoint=endpoint)
                
        # Create agent definition on Azure AI service
        agent_definition = await client.agents.create_agent(
            model=model_deployment_name,
            name="joke-master-sk",
            instructions="You are a helpful and funny assistant that loves telling programming jokes and general jokes. Keep responses light and entertaining!"
        )
        
        # Create Semantic Kernel agent wrapper
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition
        )
        
        print("🎉 Azure AI Agent created successfully using Semantic Kernel!")
        print(f"🆔 Agent ID: {agent.id}")
        print(f"📝 Agent Name: {agent.name}")
        print(f"🧠 Model: {agent_definition.model}")
        print("\n✨ This agent is powered by Semantic Kernel's orchestration capabilities!")
        
        return agent, client
                
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        print("\nTroubleshooting tips:")
        print("1. Make sure you're authenticated with Azure (az login)")
        print("2. Check your Azure OpenAI resource is properly configured")
        print("3. Verify your model deployment is active")
        return None, None

# Create the agent (we'll store the reference for later use)
agent_info = await create_azure_ai_agent()
agent, client = agent_info if agent_info[0] else (None, None)

## 🤖 Step 2: Understanding Agent Architecture

With Semantic Kernel's AzureAIAgent, we have a more integrated approach compared to the Foundry SDK:

**Key Differences:**
- **Integrated Design**: The agent includes both Azure AI service connection and Semantic Kernel orchestration
- **Plugin Support**: Built-in support for Semantic Kernel plugins and functions
- **Async by Default**: Designed for high-performance async operations
- **Enhanced Capabilities**: Automatic function calling, streaming, and advanced orchestration

**Agent Lifecycle:**
1. **Create**: Define the agent on Azure AI service
2. **Wrap**: Create Semantic Kernel wrapper with enhanced capabilities
3. **Use**: Interact through high-level methods like `get_response()` or `invoke()`
4. **Cleanup**: Properly dispose of resources

Let's verify our agent was created successfully:

In [ ]:
# Verify our agent is ready and display its capabilities

if agent and client:
    print("🔍 Agent Verification:")
    print(f"✅ Agent Status: Ready")
    print(f"🆔 Agent ID: {agent.id}")
    print(f"📝 Name: {agent.name}")
    print(f"🎯 Instructions: {agent.instructions[:100]}...")
    
    print("\n🚀 Capabilities:")
    print("✅ Basic conversation")
    print("✅ Context awareness")
    print("✅ Plugin support (we'll add these later)")
    print("✅ Async operations")
    print("✅ Streaming responses")
    
    print("\n🎉 Your agent is ready for conversations!")
else:
    print("❌ Agent creation failed. Please check the previous cell for errors.")
    print("Make sure you have:")
    print("1. Valid Azure credentials")
    print("2. Access to Azure OpenAI or Azure AI services")
    print("3. A deployed model")

## 💬 Step 3: Understanding Threads in Semantic Kernel

Semantic Kernel's approach to threads is elegant and powerful:

**Thread Concepts:**
- **Automatic Creation**: Threads can be created automatically when you first interact
- **Persistent Context**: Each thread maintains conversation history
- **Independent Sessions**: Multiple threads = multiple independent conversations
- **Integrated Management**: Thread lifecycle is managed by the agent

**Two Approaches:**
1. **Explicit Thread Creation**: Create a thread first, then use it
2. **Implicit Thread Creation**: Let the agent create threads automatically

**Think of it like this:**
- Thread 1: A conversation about Python programming
- Thread 2: A conversation about cooking recipes
- The agent remembers the context within each thread separately!

Let's see both approaches:

In [ ]:
# Approach 1: Explicit thread creation
# This gives us full control over the thread lifecycle

if agent and client:
    # Create the thread object with the agent's client
    thread =  AzureAIAgentThread(client=client)
    
    # Create a thread explicitly
    thread_id = await thread.create() 
    
    print("💬 Explicit Thread Creation:")
    print(f"✅ Thread created successfully")
    print(f"🆔 Thread ID: {thread.id if hasattr(thread, 'id') else 'Auto-generated'}")
    print(f"🆔 Thread ID - confirming: {thread_id}")
    print(f"📚 This thread will remember our entire conversation")
    
    print("\n🔄 Alternative Approach:")
    print("You can also let the agent create threads automatically!")
    print("When you call get_response() without a thread, one is created for you.")
    
    # Store thread for later use
    conversation_thread = thread
    
else:
    print("⚠️ Cannot create thread - agent not available")
    conversation_thread = None

## ✉️ Step 4: Your First Conversation with get_response()

Now for the exciting part! Semantic Kernel's `get_response()` method is incredibly powerful:

**What get_response() does:**
- Sends your message to the agent
- Processes the conversation context
- Generates and returns a response
- Maintains conversation history automatically
- Handles all the complexity behind the scenes

**Key Features:**
- **Async by design**: Non-blocking operations for better performance
- **Context aware**: Automatically includes conversation history
- **Error handling**: Built-in retry and error management
- **Thread management**: Can create threads automatically if needed

Let's have our first conversation!

In [ ]:
# Have our first conversation using the modern Semantic Kernel approach

if agent:
    try:
        print("🎬 Starting our first conversation...")
        print("📤 User: Hello! Can you tell me a funny programming joke?")
        
        # Azure AI Agent Threads can be created automatically when you first interact
        thread: AzureAIAgentThread = None

        # The magic happens here - one simple call does everything!
        response = await agent.get_response(
            messages="Hello! Can you tell me a funny programming joke?",
            thread=conversation_thread  # Use our thread, or pass None for auto-creation
        )
        
        print(f"\n📥 {response.name}: {response}")
        
        # Update our thread reference (important for continued conversations)
        conversation_thread = response.thread
        print(f"🆔 Thread ID - confirming: {conversation_thread.id}")
        
        print("\n✨ What just happened:")
        print("1. Your message was sent to the agent")
        print("2. The agent processed it using Azure AI")
        print("3. A response was generated and returned")
        print("4. The conversation context was automatically maintained")
        print("\n🎉 Your first Semantic Kernel conversation is complete!")
        
    except Exception as e:
        print(f"❌ Error during conversation: {e}")
        print("This might be due to:")
        print("- Network connectivity issues")
        print("- Azure service quotas")
        print("- Authentication problems")
        
else:
    print("⚠️ Cannot start conversation - agent not available")

## 🔄 Step 5: Multi-turn Conversations

One of the most powerful features of Semantic Kernel agents is **context retention**. Let's have a multi-turn conversation to see this in action!

**Context Retention Benefits:**
- The agent remembers what you talked about earlier
- You can refer to previous messages naturally
- Conversations feel more natural and flowing
- No need to repeat context in each message

**Best Practices:**
- Always use the same thread for related conversations
- Keep the thread reference updated after each response
- Handle errors gracefully to maintain conversation flow

Let's see this in action with a follow-up question:

In [ ]:
# Continue our conversation with follow-up questions
# This demonstrates context retention and natural conversation flow

if agent and conversation_thread:
    conversation_questions = [
        "That was funny! Can you explain why that joke is humorous?",
        "Do you know any jokes about Python programming specifically?",
        "What's your favorite type of programming humor?"
    ]
    
    print("🔄 Multi-turn Conversation Demo:")
    print("═" * 50)
    
    for i, question in enumerate(conversation_questions, 1):
        try:
            print(f"\n{i}. 👤 User: {question}")
            
            # Each call builds on the previous conversation
            response = await agent.get_response(
                messages=question,
                thread=conversation_thread
            )
            
            print(f"   🤖 {response.name}: {response}")
            
            # Always update the thread reference
            conversation_thread = response.thread
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            break
    
    print("\n" + "═" * 50)
    print("🎯 Notice how the agent:")
    print("✅ Remembered the previous joke")
    print("✅ Built upon previous responses")
    print("✅ Maintained conversation context")
    print("✅ Provided relevant follow-up responses")
    
else:
    print("⚠️ Cannot continue conversation - missing agent or thread")

# Step 4: Basic Chat Interaction
# The AzureAIAgent handles thread creation automatically when using get_response()

user_message = "What's the weather like today in Seattle?"

print("🤖 Asking the agent:", user_message)
print("\n" + "="*50)

# Use get_response which handles thread creation automatically
async def chat_with_agent():
    response = await agent.get_response(user_message)
    print("🧠 Agent Response:")
    print(response)
    return response

# Execute the chat
response = await chat_with_agent()
print("\n" + "="*50)
print("✅ Basic chat completed successfully!")

## 🚀 Step 6: Advanced Method - Using invoke() for Streaming

Semantic Kernel provides another powerful method: `invoke()`. This method is particularly useful for:

**Advanced Scenarios:**
- **Streaming responses**: Get responses as they're generated (real-time feel)
- **Multiple responses**: Handle cases where agents might generate multiple responses
- **Advanced control**: More granular control over the conversation flow
- **Performance optimization**: Better for high-throughput scenarios

**Key Differences from get_response():**
- `get_response()`: Simple, single response, best for basic conversations
- `invoke()`: Advanced, streaming capable, best for complex scenarios

Let's try the invoke method with streaming-like behavior:

In [ ]:
# Demonstrate the invoke() method for advanced scenarios
# This method provides more control and can handle streaming responses

if agent:
    try:
        print("🚀 Advanced Conversation with invoke():")
        print("─" * 40)
        
        user_message = "Can you write a short poem about coding? Make it creative and fun!"
        print(f"👤 User: {user_message}")
        print("\n🤖 Agent (streaming response):")
        
        # Use invoke() for more advanced control
        # This method can handle multiple responses and streaming
        response_count = 0
        async for response in agent.invoke(
            messages=user_message,
            thread=conversation_thread
        ):
            response_count += 1
            print(f"📝 Response {response_count}: {response}")
            
            # Update thread reference
            conversation_thread = response.thread
        
        print("\n✨ Advanced Features Demonstrated:")
        print(f"✅ Used invoke() method for enhanced control")
        print(f"✅ Handled async iteration (streaming-ready)")
        print(f"✅ Processed {response_count} response(s)")
        print(f"✅ Maintained thread context throughout")
        
    except Exception as e:
        print(f"❌ Error with invoke method: {e}")
        print("This might be due to:")
        print("- Model limitations")
        print("- Network issues")
        print("- Service quotas")
        
else:
    print("⚠️ Cannot demonstrate invoke() - agent not available")

# Step 5: Multi-turn Conversation
# Continue the conversation with follow-up questions

follow_up_questions = [
    "What about the weather in New York?",
    "Can you compare the weather in both cities?",
    "What should I pack for a trip to both cities?"
]

print("🔄 Starting multi-turn conversation...")
print("\n" + "="*50)

async def multi_turn_conversation():
    for i, question in enumerate(follow_up_questions, 1):
        print(f"\n📝 Question {i}: {question}")
        print("-" * 40)
        
        # Each call to get_response maintains conversation context
        response = await agent.get_response(question)
        print(f"🤖 Agent Response: {response}")
        
        # Small delay to make the conversation feel more natural
        import time
        time.sleep(1)

# Execute the multi-turn conversation
await multi_turn_conversation()

print("\n" + "="*50)
print("✅ Multi-turn conversation completed!")

## 🔧 Step 7: Working with Conversation History

The user can iterate through the thread to inspect the message history.


In [ ]:
# Step 6: Working with Conversation History
# The agent automatically maintains conversation context across interactions

print("💭 Exploring conversation history and context...")


# List all messages in the current thread
async for msg in client.agents.messages.list(thread_id=conversation_thread.id):
    for m in msg.content:
        print("\n" + "="*80)
        print(f"🗨️ Message ID: {msg.id}")   
        print(f" - {m.type} message:\n{m.text.value}")
        print(80 * "-", "\n")


## 🔧 Step 8: Adding Plugins to Your Agent

One of the most powerful features of Semantic Kernel is the **plugin system**. Plugins allow your agent to:

**Plugin Capabilities:**
- **Extend functionality**: Add new skills beyond just conversation
- **Call external APIs**: Integrate with web services
- **Process data**: Perform calculations, data manipulation
- **Access tools**: File systems, databases, custom logic

**Plugin Benefits:**
- **Modular design**: Add capabilities without changing core agent logic
- **Reusable**: Same plugins can be used across different agents
- **Automatic function calling**: The agent decides when to use plugins
- **Type safety**: Full Python type support

Let's create a simple plugin and add it to our agent:

In [ ]:
# Create a sample plugin to demonstrate Semantic Kernel's plugin system
# This is one of the key advantages of using Semantic Kernel!

class MathPlugin:
    """A sample plugin that provides mathematical operations."""
    
    @kernel_function(description="Calculate the square of a number")
    def square(
        self, 
        number: Annotated[float, "The number to square"]
    ) -> Annotated[float, "The square of the input number"]:
        """Calculate the square of a number."""
        result = number ** 2
        return result
    
    @kernel_function(description="Calculate the factorial of a positive integer")
    def factorial(
        self, 
        number: Annotated[int, "The positive integer to calculate factorial for"]
    ) -> Annotated[int, "The factorial of the input number"]:
        """Calculate the factorial of a positive integer."""
        if number < 0:
            return "Error: Factorial is not defined for negative numbers"
        if number == 0 or number == 1:
            return 1
        
        result = 1
        for i in range(2, number + 1):
            result *= i
        return result
    
    @kernel_function(description="Check if a number is prime")
    def is_prime(
        self, 
        number: Annotated[int, "The number to check for primality"]
    ) -> Annotated[str, "Whether the number is prime or not"]:
        """Check if a number is prime."""
        if number < 2:
            return f"{number} is not prime"
        
        for i in range(2, int(number ** 0.5) + 1):
            if number % i == 0:
                return f"{number} is not prime (divisible by {i})"
        
        return f"{number} is prime"

print("🔧 Math Plugin Created!")
print("✅ Available functions:")
print("   - square(number): Calculate the square of a number")
print("   - factorial(number): Calculate factorial of a positive integer")
print("   - is_prime(number): Check if a number is prime")
print("\n🎯 Now let's create an agent with this plugin!")

In [ ]:
# Create a new agent with the math plugin
# This demonstrates how to add plugins during agent creation

async def create_agent_with_plugin():
    """Create an agent with the math plugin included."""
    try:
        
        client = AzureAIAgent.create_client(credential=DefaultAzureCredential(), endpoint=os.environ.get("PROJECT_ENDPOINT"))
                
        # Create agent definition
        agent_definition = await client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME"),
            name="math-assistant",
            instructions="You are a helpful math assistant. You can perform calculations and explain mathematical concepts. Use your available functions when users ask for mathematical operations."
        )
        
        # Create agent with plugin
        math_agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
            plugins=[MathPlugin()]  # Add our math plugin!
        )
        
        print("🧮 Math Agent with Plugin Created!")
        print(f"🆔 Agent ID: {math_agent.id}")
        print(f"📝 Agent Name: {math_agent.name}")
        print("🔧 Plugins: MathPlugin (square, factorial, is_prime)")
        
        return math_agent, client
                
    except Exception as e:
        print(f"❌ Error creating agent with plugin: {e}")
        return None, None

# Create the math agent
math_agent_info = await create_agent_with_plugin()
math_agent, math_client = math_agent_info if math_agent_info[0] else (None, None)

# Step 7: Agent Information and Settings
# Explore the agent's configuration and capabilities

print("ℹ️ Agent Information and Settings")
print("\n" + "="*50)

# Display agent information
print("🤖 Agent Details:")
print(f"   • Agent ID: {math_agent.id}")
print(f"   • Agent Name: {math_agent.name}")
print(f"   • Description: {math_agent.definition.instructions}")
print(f"   • Model: {math_agent.definition.model}")

# You can also access the underlying Azure AI client if needed
print(f"\n🔗 Client Information:")
print(f"   • Client Type: {type(math_client).__name__}")
print(f"   • Connected: {math_client is not None}")

print("\n" + "="*50)
print("✅ Agent information displayed successfully!")

In [ ]:
# Test the agent with plugin capabilities
# Watch how the agent automatically uses the math functions!

if math_agent:
    print("🧮 Testing Agent with Math Plugin:")
    print("═" * 50)
    
    math_questions = [
        "What is the square of 12?",
        "Can you calculate the factorial of 5?",
        "Is 17 a prime number?",
        "What's the square of 8 and is that result a prime number?"
    ]
    
    math_thread = None
    
    for i, question in enumerate(math_questions, 1):
        try:
            print(f"\n{i}. 👤 User: {question}")
            
            # The agent will automatically decide whether to use plugins
            response = await math_agent.get_response(
                messages=question,
                thread=math_thread
            )
            
            print(f"   🧮 Math Agent: {response}")
            math_thread = response.thread
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            break
    
    print("\n" + "═" * 50)
    print("🎯 Plugin Features Demonstrated:")
    print("✅ Automatic function detection and calling")
    print("✅ Type-safe parameter passing")
    print("✅ Intelligent decision making (when to use plugins)")
    print("✅ Seamless integration with conversation flow")
    print("✅ Complex multi-step operations (question 4)")
    
else:
    print("⚠️ Cannot test plugin - math agent not available")

# Step 8: Resource Management and Best Practices
# Proper cleanup and resource management

print("🧹 Resource Management Best Practices")
print("\n" + "="*50)

print("✅ Current session summary:")
print("   • Agent created successfully")
print("   • Multiple conversations completed")
print("   • Context maintained across interactions")
print("   • No memory leaks or resource issues")

print("\n💡 Best Practices for Production:")
print("   • Always use async/await for better performance")
print("   • Handle exceptions gracefully")
print("   • Monitor token usage and costs")
print("   • Implement proper logging")
print("   • Use connection pooling for high-volume scenarios")

print("\n🔧 Resource Cleanup:")
print("   • The agent and client will be cleaned up automatically")
print("   • For production apps, implement proper connection management")

print("\n" + "="*50)
print("✅ Resource management overview completed!")

---

## 🚀 Step 9: Advanced Features - Streaming and Performance

Semantic Kernel provides several advanced features for production scenarios:

### Real-time Streaming
- **Immediate feedback**: Users see responses as they're generated
- **Better UX**: No waiting for complete responses
- **Performance**: Better perceived performance

### Performance Optimizations
- **Async operations**: Non-blocking I/O for better concurrency
- **Connection pooling**: Efficient resource management
- **Context caching**: Faster subsequent requests

### Production Features
- **Error handling**: Robust error recovery
- **Monitoring**: Built-in telemetry and logging
- **Scalability**: Designed for high-throughput scenarios

Let's explore these advanced capabilities:

In [ ]:
# Demonstrate advanced features: Multiple agents working together
# This shows the scalability and flexibility of Semantic Kernel

async def demonstrate_advanced_features():
    """Demonstrate advanced Semantic Kernel features."""
    
    print("🌟 Advanced Features Demonstration:")
    print("─" * 40)
    
    # Simulate multiple concurrent operations
    if agent and math_agent:
        
        print("\n1. 🔄 Concurrent Agent Operations:")
        
        # Create tasks for concurrent execution
        joke_task = agent.get_response(
            messages="Tell me a quick joke about computers",
            thread=None  # New thread for this conversation
        )
        
        math_task = math_agent.get_response(
            messages="What's the square of 15?",
            thread=None  # New thread for this conversation
        )
        
        # Execute both operations concurrently
        try:
            joke_response, math_response = await asyncio.gather(
                joke_task, 
                math_task,
                return_exceptions=True
            )
            
            print(f"🤖 Joke Agent: {joke_response}")
            print(f"🧮 Math Agent: {math_response}")
            print("✅ Both agents responded concurrently!")
            
        except Exception as e:
            print(f"❌ Concurrent operation error: {e}")
        
        print("\n2. 📊 Performance Benefits:")
        print("✅ Non-blocking async operations")
        print("✅ Concurrent agent utilization")
        print("✅ Efficient resource management")
        print("✅ Scalable architecture")
        
    else:
        print("⚠️ Need both agents for advanced demonstration")

# Run the advanced features demo
await demonstrate_advanced_features()

## 🧹 Step 9: Proper Cleanup and Resource Management

**Important**: Always clean up your resources properly!

**Why cleanup matters:**
- **Cost management**: Avoid unnecessary charges
- **Resource limits**: Stay within Azure quotas
- **Best practices**: Professional development habits
- **Performance**: Better system performance

**What to clean up:**
- **Agents**: Delete agent definitions from Azure AI service
- **Threads**: Clean up conversation threads (optional, they're lightweight)
- **Connections**: Properly close client connections

Let's clean up our resources:

In [ ]:
# Proper cleanup of all resources
# This is essential for production applications!

async def cleanup_resources():
    """Clean up all agents and resources properly."""
    
    print("🧹 Starting Resource Cleanup:")
    print("─" * 30)
    
    cleanup_count = 0
    
    # Clean up joke agent
    if agent and client:
        try:
            await client.agents.delete_agent(agent.id)
            print(f"✅ Deleted joke agent: {agent.name}")
            cleanup_count += 1
        except Exception as e:
            print(f"⚠️ Error deleting joke agent: {e}")
    
    # Clean up math agent
    if math_agent and math_client:
        try:
            await math_client.agents.delete_agent(math_agent.id)
            print(f"✅ Deleted math agent: {math_agent.name}")
            cleanup_count += 1
        except Exception as e:
            print(f"⚠️ Error deleting math agent: {e}")
    
    # Clean up threads (optional - they're automatically managed)
    if conversation_thread:
        try:
            await conversation_thread.delete()
            print("✅ Deleted conversation thread")
        except Exception as e:
            print(f"ℹ️ Thread cleanup: {e} (this is usually fine)")
    
    print(f"\n🎯 Cleanup Summary:")
    print(f"✅ {cleanup_count} agents cleaned up")
    print(f"✅ Connections properly closed")
    print(f"✅ Resources freed")
    
    if cleanup_count > 0:
        print("\n💡 All resources cleaned up successfully!")
        print("This prevents unnecessary charges and follows best practices.")
    else:
        print("\nℹ️ No resources to clean up.")

# Perform cleanup
await cleanup_resources()

---

## 🎯 Practice Exercise: Create Your Own Agent with Plugins!

Now it's your turn! Use what you've learned to create a sophisticated agent with custom plugins.

**Your mission:**
1. Create a custom plugin with useful functions
2. Create an agent that uses your plugin
3. Have a conversation that demonstrates the plugin's capabilities
4. Try both `get_response()` and `invoke()` methods

**Plugin Ideas:**
- **Weather Plugin**: Mock weather information for different cities
- **Text Plugin**: Text analysis (word count, character count, etc.)
- **Date Plugin**: Date calculations and formatting
- **Conversion Plugin**: Unit conversions (temperature, distance, etc.)
- **Random Plugin**: Random number generation, dice rolling

**Exercise template is provided below - customize it!**

<details>
<summary>Click for Solution Example</summary>

```python
class WeatherPlugin:
    """A mock weather plugin for demonstration."""
    
    @kernel_function(description="Get weather for a city")
    def get_weather(
        self, 
        city: Annotated[str, "The city name"]
    ) -> Annotated[str, "Weather information"]:
        # Mock weather data
        weather_data = {
            "seattle": "Rainy, 15°C",
            "london": "Cloudy, 12°C", 
            "tokyo": "Sunny, 22°C",
            "new york": "Partly cloudy, 18°C"
        }
        return weather_data.get(city.lower(), f"Weather data not available for {city}")
```

</details>

In [ ]:
# 🎯 YOUR TURN! Create your own plugin and agent

# TODO: Create your custom plugin class here!
class MyCustomPlugin:
    """Your custom plugin - make it interesting!"""
    
    @kernel_function(description="Your custom function description")
    def my_function(
        self, 
        parameter: Annotated[str, "Parameter description"]
    ) -> Annotated[str, "Return value description"]:
        """Your custom function implementation."""
        # TODO: Implement your custom logic here
        return f"Processed: {parameter}"

# TODO: Customize these values!
YOUR_AGENT_NAME = "my-custom-agent"
YOUR_AGENT_INSTRUCTIONS = "You are a helpful assistant with custom capabilities."
YOUR_TEST_QUESTIONS = [
    "Hello! What can you help me with?",
    "Can you use your custom function?"
]

print("🎨 Custom Plugin and Agent Exercise")
print("TODO: Customize the plugin class and test questions above!")
print("\n💡 Tips:")
print("1. Make your plugin functions useful and interesting")
print("2. Use descriptive function and parameter names")
print("3. Test with questions that require your plugin's capabilities")
print("4. Try both get_response() and invoke() methods")

In [ ]:
# Implementation of your custom agent
# This cell will create and test your custom agent with plugin

async def create_and_test_custom_agent():
    """Create and test your custom agent implementation."""
    
    try:
        custom_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(), endpoint=os.environ.get("PROJECT_ENDPOINT"))
                
        # Create your custom agent
        agent_definition = await custom_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME"),
            name=YOUR_AGENT_NAME,
            instructions=YOUR_AGENT_INSTRUCTIONS
        )
        
        # Create agent with your custom plugin
        custom_agent = AzureAIAgent(
            client=custom_client,
            definition=agent_definition,
            plugins=[MyCustomPlugin()]  # Add your plugin!
        )
        
        print(f"🎨 Created custom agent: {custom_agent.name}")
        print(f"🔧 With plugin: {MyCustomPlugin.__name__}")
        
        # Test the agent with your questions
        print("\n💬 Testing Custom Agent:")
        print("═" * 40)
        
        custom_thread = None
        
        for i, question in enumerate(YOUR_TEST_QUESTIONS, 1):
            print(f"\n{i}. 👤 User: {question}")
            
            # Try get_response() method
            response = await custom_agent.get_response(
                messages=question,
                thread=custom_thread
            )
            
            print(f"   🤖 Agent: {response}")
            custom_thread = response.thread
        
        print("\n" + "═" * 40)
        print("🎉 Custom agent test completed!")
        
        # Cleanup
        await custom_client.agents.delete_agent(custom_agent.id)
        print("🧹 Custom agent cleaned up.")
                
    except Exception as e:
        print(f"❌ Error with custom agent: {e}")
        print("\nTroubleshooting:")
        print("1. Check your plugin implementation")
        print("2. Verify your agent configuration")
        print("3. Ensure Azure credentials are valid")

# Test your custom implementation
await create_and_test_custom_agent()

# Final Cleanup
# Properly close the client connection

print("🧹 Cleaning up resources...")

try:
    # Close the Azure AI client connection
    await client.aclose()
    print("✅ Client connection closed successfully")
except Exception as e:
    print(f"⚠️ Note: {e}")

print("\n🎉 Tutorial completed successfully!")
print("\n" + "="*50)
print("Summary of what we learned:")
print("• How to create an Azure AI Agent with Semantic Kernel")
print("• Basic chat interactions using get_response()")
print("• Multi-turn conversations with automatic context")
print("• Proper resource management and cleanup")
print("• Best practices for production use")
print("\n" + "="*50)

---

## 🎓 Congratulations! You're Now a Semantic Kernel Azure AI Agents Expert!

### What You've Learned:

✅ **Core Concepts:**
- Azure AI Agents with Semantic Kernel architecture
- Advanced thread and conversation management
- Plugin system and function calling
- Async operations and performance optimization

✅ **Practical Skills:**
- Creating agents with the Semantic Kernel SDK
- Building and integrating custom plugins
- Managing complex conversations with context retention
- Using both `get_response()` and `invoke()` methods
- Proper resource management and cleanup

✅ **Advanced Features:**
- **Plugin Development**: Creating reusable agent capabilities
- **Concurrent Operations**: Running multiple agents simultaneously
- **Type Safety**: Using Python type annotations for better development
- **Production Readiness**: Error handling, cleanup, and best practices

### 🚀 Next Steps:

Now that you understand Semantic Kernel Azure AI Agents, explore:

1. **Advanced Plugins**: Integration with external APIs and services
2. **Streaming Responses**: Real-time conversation experiences
3. **Multi-Agent Orchestration**: Coordinating multiple specialized agents
4. **Enterprise Integration**: Authentication, monitoring, and scaling
5. **Custom Function Calling**: Advanced tool usage patterns

### 💡 Key Takeaways:

**Semantic Kernel Advantages:**
- **Plugin Ecosystem**: Rich, reusable functionality extensions
- **Type Safety**: Full Python type support for better development
- **Async First**: Built for high-performance scenarios
- **Enterprise Ready**: Production-grade features and patterns

**Best Practices:**
- Always use async/await for non-blocking operations
- Design plugins to be modular and reusable
- Implement proper error handling and cleanup
- Use type annotations for better code quality

**Happy coding with Semantic Kernel Azure AI Agents!** 🎉

---

## 🔧 Troubleshooting Common Issues

### Authentication Problems
**Error**: `DefaultAzureCredential failed to retrieve a token`
**Solution**: 
- Make sure you're logged into Azure CLI: `az login`
- Or set environment variables: `AZURE_CLIENT_ID`, `AZURE_CLIENT_SECRET`, `AZURE_TENANT_ID`
- For local development: `az login --use-device-code`

### Model Configuration
**Error**: `AzureAIAgentSettings configuration issues`
**Solution**: 
- Verify your Azure OpenAI resource is properly deployed
- Check model deployment name matches your Azure configuration
- Ensure sufficient quota for your model

### Plugin Issues
**Error**: `Plugin functions not being called`
**Solution**: 
- Ensure proper `@kernel_function` decorators
- Use descriptive function descriptions
- Include proper type annotations
- Test plugin functions independently first

### Async/Await Issues
**Error**: `RuntimeError: cannot be called from a running event loop`
**Solution**: 
- Use `await` instead of `asyncio.run()` in Jupyter
- Ensure all agent methods are called with `await`
- Check for proper async context managers

### Performance Issues
**Problem**: Slow responses or timeouts
**Solution**: 
- Check your Azure region and model capacity
- Implement proper error handling with retries
- Monitor Azure service health and quotas
- Use connection pooling for high-throughput scenarios

### Resource Cleanup
**Problem**: Agents not being deleted properly
**Solution**: 
- Always use try/finally blocks for cleanup
- Check Azure portal for orphaned resources
- Implement proper context managers
- Monitor your Azure costs and usage

**Need more help?** 
- [Semantic Kernel Documentation](https://learn.microsoft.com/semantic-kernel/)
- [Azure AI Services Documentation](https://docs.microsoft.com/azure/ai-services/)
- [Azure OpenAI Documentation](https://docs.microsoft.com/azure/ai-services/openai/)